In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
import tensorflow as tf
from keras import regularizers
from keras import optimizers
from sklearn.metrics import roc_curve
from keras import backend as K


from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model


warnings.filterwarnings('ignore')

Предобработка данных

In [22]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
labels = 'human spoof'.split()
for g in labels:
    pathlib.Path(f'C:/Audiodata/Training_Data/img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'C:/Audiodata/Training_Data/{g}'):
        name = f'C:/Audiodata/Training_Data/{g}/{filename}'
        y, sr = librosa.load(name, mono=True)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'C:/Audiodata/Training_Data/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
        
        
        
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
#genres = 'sample'.split()
pathlib.Path(f'C:/Audiodata/Testing_Data_MLTASK/img_data').mkdir(parents=True, exist_ok=True)
for filename in os.listdir(f'C:/Audiodata//Testing_Data_MLTASK/Testing_Data/'):
    name = f'C:/Audiodata//Testing_Data_MLTASK/Testing_Data/{filename}'
    y, sr = librosa.load(name, mono=True)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig(f'C:/Audiodata/Testing_Data_MLTASK/img_data/{filename[:-3].replace(".", "")}.png')
    plt.clf()

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

Извлечение признаков

In [23]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
labels = 'human spoof'.split()
for g in labels:
    for filename in os.listdir(f'C:/Audiodata/Training_Data/{g}'):
        name = f'C:/Audiodata/Training_Data/{g}/{filename}'
        y, sr = librosa.load(name, mono=True)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
            
            
            
            
file = open('dataset_testing.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

for filename in os.listdir(f'C:/Audiodata//Testing_Data_MLTASK/Testing_Data/'):
    name = f'C:/Audiodata//Testing_Data_MLTASK/Testing_Data/{filename}'
    y, sr = librosa.load(name, mono=True)
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open('dataset_testing.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

In [4]:
data.shape

(50000, 28)

In [5]:
data = pd.read_csv('dataset.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,human_00000.wav,0.353022,0.042762,1698.311629,1708.273546,3365.404926,0.087067,-368.347534,113.735191,12.970050,...,-4.520185,-4.032321,-8.738158,-4.378026,-13.430143,4.926398,-7.972149,-5.450689,-2.770082,human
1,human_00001.wav,0.479716,0.003636,1553.135885,1514.208772,2904.209206,0.067834,-562.947815,121.397758,-18.131916,...,8.148829,11.660575,-4.594286,14.093517,-1.010832,7.608933,2.417472,12.535057,2.939179,human
2,human_00002.wav,0.345533,0.059232,1458.539398,1590.756452,2949.948674,0.062064,-348.697754,116.993080,6.470922,...,-0.932909,-4.581821,1.384029,1.174273,-5.940725,2.146937,-7.342173,-3.320397,-2.511549,human
3,human_00003.wav,0.312623,0.004049,2377.953299,1805.998107,4344.411981,0.161133,-538.822083,85.091057,-46.547985,...,-12.554141,-3.270665,-10.091649,2.089070,-10.832202,-0.316745,-10.121891,-2.006852,-9.425464,human
4,human_00004.wav,0.397256,0.014278,1932.324862,1383.286507,3284.746582,0.138356,-426.593384,125.052292,-51.205925,...,-4.455950,-0.867689,-8.473681,4.679374,-8.171278,-0.674228,-3.593828,-2.961659,-3.126526,human


In [6]:
#Создание меток
label_list = data.iloc[:, -1]
label_list=label_list.mask(label_list == 'spoof', 0)
label_list=label_list.mask(label_list == 'human', 1)

# Масштабирование столбцов признаков

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, 1:-1], dtype = float))
label_list=label_list.astype(float)

# Разделение данных на обучающий,валидационный и тестовый набор
X_temp, X_valid, y_temp, y_valid = train_test_split(X, label_list, test_size=0.2, stratify=label_list, random_state=20)
X_train, X_test, y_train, y_test = train_test_split(X_temp,  y_temp, test_size=0.2,stratify=y_temp, random_state=20)

In [7]:
data_testing = pd.read_csv('dataset_testing.csv')
data_testing=data_testing.drop(columns=['label'])
data_testing.head()

# Масштабирование столбцов признаков
X_testing = scaler.transform(np.array(data_testing.iloc[:, 1:], dtype = float))

In [8]:
def equal_error_rate(y_true, y_pred):
    n_imp = tf.math.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.math.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.math.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.math.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

In [29]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=[equal_error_rate])
model.summary()

model.fit(X_train,
            y_train,
            epochs=100,
            batch_size=256,
            validation_data=(X_valid,y_valid))

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 256)               6912      
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 129       
Total params: 39,937
Trainable params: 39,937
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
125/125 [==============================] - 23s 179ms/step - loss: 0.3617 - equal_error_rate: 0.2382 - v

125/125 [==============================] - 26s 206ms/step - loss: 0.0847 - equal_error_rate: 0.0425 - val_loss: 0.0706 - val_equal_error_rate: 0.0364
Epoch 46/100
125/125 [==============================] - 25s 202ms/step - loss: 0.0798 - equal_error_rate: 0.0407 - val_loss: 0.0715 - val_equal_error_rate: 0.0370
Epoch 47/100
125/125 [==============================] - 25s 201ms/step - loss: 0.0802 - equal_error_rate: 0.0401 - val_loss: 0.0702 - val_equal_error_rate: 0.0373
Epoch 48/100
125/125 [==============================] - 25s 200ms/step - loss: 0.0773 - equal_error_rate: 0.0390 - val_loss: 0.0711 - val_equal_error_rate: 0.0357
Epoch 49/100
125/125 [==============================] - 25s 197ms/step - loss: 0.0787 - equal_error_rate: 0.0402 - val_loss: 0.0700 - val_equal_error_rate: 0.0369
Epoch 50/100
125/125 [==============================] - 26s 206ms/step - loss: 0.0767 - equal_error_rate: 0.0382 - val_loss: 0.0676 - val_equal_error_rate: 0.0349
Epoch 51/100
125/125 [=============

125/125 [==============================] - 27s 216ms/step - loss: 0.0588 - equal_error_rate: 0.0291 - val_loss: 0.0623 - val_equal_error_rate: 0.0328
Epoch 96/100
125/125 [==============================] - 26s 209ms/step - loss: 0.0601 - equal_error_rate: 0.0320 - val_loss: 0.0606 - val_equal_error_rate: 0.0332
Epoch 97/100
125/125 [==============================] - 25s 197ms/step - loss: 0.0602 - equal_error_rate: 0.0314 - val_loss: 0.0610 - val_equal_error_rate: 0.0334
Epoch 98/100
125/125 [==============================] - 26s 210ms/step - loss: 0.0558 - equal_error_rate: 0.0282 - val_loss: 0.0599 - val_equal_error_rate: 0.0318
Epoch 99/100
125/125 [==============================] - 25s 200ms/step - loss: 0.0607 - equal_error_rate: 0.0307 - val_loss: 0.0608 - val_equal_error_rate: 0.0333
Epoch 100/100
125/125 [==============================] - 26s 210ms/step - loss: 0.0603 - equal_error_rate: 0.0303 - val_loss: 0.0627 - val_equal_error_rate: 0.0328


In [31]:
model_predict=model.predict(X_test, batch_size=128)
print('Test_EER=',equal_error_rate(y_test, model_predict))

Test_EER= tf.Tensor(0.02631341, shape=(), dtype=float32)


In [32]:
model_predict_testing=model.predict(X_testing, batch_size=128)

In [33]:
answer=pd.DataFrame(data_testing[['filename']])
answer['score']=pd.DataFrame(model_predict_testing)

In [36]:
answer.head()

,filename,score
0,sample_0000.wav,0.557822
1,sample_0001.wav,0.000389
2,sample_0002.wav,0.874096
3,sample_0003.wav,0.832074
4,sample_0004.wav,0.972879


In [38]:
answer.to_csv(path_or_buf='D:\\Test_Result.csv',index=False)